In [136]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from copy import copy

In [112]:
# Open the data file
File_Path = '/Users/nv83/Dropbox/Research/Vacanti_Laboratory/projects/Alzheimers/MouseAstrocytes_oAB_INSRinhibition/inEFP/Astrocytes_inEFP.AIA/RelativeFluxes.xlsx'
RawData = pd.read_excel(File_Path, sheet_name='MIDs')

# Set the row names
RawData.index = RawData.loc[:,'Unnamed: 0']
RawData.drop('Unnamed: 0', axis=1, inplace=True)

# Set the column names and drop unwanted columns
RawData.columns = RawData.iloc[1,:]
ColumsToDrop = ['fm3-2','theoretical','nothing']
[RawData.drop(column,axis=1, inplace=True) for column in ColumsToDrop]


# Filter out unneeded rows
# Drop rows with nothing in them
RawData = RawData[RawData.index.notnull()]

# Get the row location of where the MIDs and Peak Areas start
MID_index = RawData.index.get_loc('MIDs')
AreaIndex = RawData.index.get_loc('Peak Areas (ion counts * seconds)')

# Inidcate which fragments need the peak areas to compute ratios
SpecialFragments = ['lactate_233', 'lactate_261', 'HPA_261']

# Perform the row filtering
RowsToKeep = [((i>MID_index) | (RawData.index[i] in SpecialFragments)) & (i>AreaIndex) for i in range(len(RawData.index))]
RawData = RawData[RowsToKeep]

In [126]:
# Create a data frame to contain the ratios of metabolites to internal standards
Ratios = pd.DataFrame(index=RawData.index, columns=RawData.columns)

# Remove rows corresponding to the peak areas for fragments that rely on peak areas
[Ratios.drop(fragment, axis=0, inplace=True) for fragment in SpecialFragments]

# Rename the rows to correspond to fragments (i.e. remove the M0 or M1)
Ratios.index = [i.split(' ')[0] for i in Ratios.index]

# Drop duplicate row names - each fragment had a name that came from the M0 and M1 forms of the fragment
Ratios = Ratios[~Ratios.index.duplicated(keep='first')]

# After removing the M0 and M1, the row corresponding to the internal standard for lactate needs to be removed
Ratios.drop('HPA_261', axis=0, inplace=True)

# Compute the ratios for lactate independently because it relies on peak area ratios
Ratios.loc['lactate_233',:] = RawData.loc['lactate_233',:]/RawData.loc['HPA_261',:]
Ratios.loc['lactate_261',:] = RawData.loc['lactate_261',:]/RawData.loc['HPA_261',:]

# Compute the ratios for the remaining fragments
for i in Ratios.index:
    if i not in SpecialFragments:
        numerator = RawData.loc[i+' M0',:]
        denominator = RawData.loc[i+ ' M1',:]
        Ratios.loc[i,:] = np.divide(numerator.to_numpy(), denominator.to_numpy(), out=np.full_like(numerator.to_numpy(), np.nan), where=denominator.to_numpy()!=0)

In [182]:
MetaData = pd.read_excel(File_Path, sheet_name='MetaData')
Selected_Fragments = np.array(pd.read_excel(File_Path, sheet_name='SelectFragments',header=None).iloc[:,0])
Selected_Fragments

array(['lactate_233', 'alanine_260', 'glycine_246', 'valine_288',
       'leucine_274', 'isoleucine_274', 'serine_390', 'threonine_404',
       'methionine_320', 'phenylalanine_336', 'aspartate_418',
       'glutamate_432', 'asparagine_417', 'lysine_431', 'glutamine_431',
       'arginine_442', 'tyrosine_302', 'histidine_440', 'tryptophan_302'],
      dtype=object)

In [175]:
FreshMediaSamples = np.array(MetaData.loc[MetaData.loc[:,'Time (Days)']==0,'Sample'])
FreshMedia = copy(Ratios.loc[:, [column in FreshMediaSamples for column in Ratios.columns]])
FreshMedia_RowMeanNorm = FreshMedia.apply(lambda row: row / row.mean(), axis=1)
FreshMedia_RMN_ColMed = [np.median(FreshMedia_RowMeanNorm.loc[:,column]) for column in FreshMedia_RowMeanNorm.columns]
FreshMedia_ColMedNorm = FreshMedia.div(FreshMedia_RMN_ColMed,axis='columns')

SpentMediaSamples_Day2 = np.array(MetaData.loc[MetaData.loc[:,'Time (Days)']==2,'Sample'])
SpentMediaDay2 = copy(Ratios.loc[:, [column in SpentMediaSamples_Day2 for column in Ratios.columns]])
SpentMediaDay2_RowMeanNorm = SpentMediaDay2.apply(lambda row: row / row.mean(), axis=1)
SpentMediaDay2_RMN_ColMed = [np.median(SpentMediaDay2_RowMeanNorm.loc[:,column]) for column in SpentMediaDay2_RowMeanNorm.columns]
SpentMediaDay2_ColMedNorm = SpentMediaDay2.div(SpentMediaDay2_RMN_ColMed,axis='columns')

FreshMediaAverages = [np.mean(FreshMedia_ColMedNorm.loc[fragment,:]) for fragment in FreshMedia_ColMedNorm.index]

Delta = SpentMediaDay2_ColMedNorm.sub(FreshMediaAverages,axis='index')
# select fragments before normalization






In [176]:
Delta

,sm57,sm58,sm59,sm60,sm61,sm62,sm63,sm64,sm65,sm66,...,sm71,sm72,sm73,sm74,sm75,sm76,sm77,sm78,sm79,sm80
lactate_233,2.682935,2.180238,1.880331,2.955046,3.321419,3.668491,3.274955,3.99367,3.154705,2.450336,...,3.328555,2.765025,2.140366,3.467579,3.669289,2.405065,3.378327,2.591783,3.978954,3.962989
lactate_261,3.89285,3.082101,2.699922,4.3139,4.889516,5.358715,4.850441,5.920149,4.659707,3.50964,...,4.978924,4.083694,3.10416,5.057346,5.449582,3.447487,4.950461,3.719696,5.812888,5.787917
alanine_232,0.150461,0.184548,0.158294,0.182772,0.155752,0.187181,0.169828,0.253129,0.18194,0.24137,...,0.169386,0.233588,0.148342,0.223231,0.190011,0.237667,0.182573,0.245503,0.193188,0.280565
alanine_260,0.14736,0.187759,0.157681,0.173592,0.150057,0.175154,0.174801,0.24774,0.168008,0.239826,...,0.149295,0.224999,0.138076,0.221677,0.166389,0.220746,0.16431,0.223815,0.184379,0.254514
glycine_246,-0.11372,-0.135787,-0.104298,-0.154978,-0.117351,-0.143252,-0.126381,-0.160745,-0.127728,-0.13587,...,-0.068826,-0.100479,-0.078179,-0.105354,-0.138959,-0.165626,-0.120383,-0.161986,-0.132158,-0.180039
glycine_218,-0.109628,-0.130128,-0.096717,-0.135839,-0.10098,-0.13658,-0.119463,-0.154383,-0.117295,-0.117238,...,-0.055223,-0.101428,-0.072677,-0.092485,-0.126752,-0.149532,-0.121778,-0.138608,-0.114472,-0.172584
valine_288,0.048405,0.037098,0.041181,0.040707,0.052887,0.04022,0.051891,0.050872,0.046069,0.043043,...,0.059354,0.046573,0.06419,0.04697,0.055127,0.041835,0.052124,0.042828,0.063131,0.044804
valine_260,0.044102,0.037466,0.041835,0.043101,0.046681,0.037225,0.05175,0.042471,0.04115,0.04021,...,0.05507,0.04033,0.048082,0.048743,0.047484,0.045534,0.050739,0.041185,0.062842,0.046839
leucine_274,-0.085824,-0.112967,-0.069118,-0.111063,-0.086657,-0.11751,-0.04337,-0.081874,-0.060011,-0.078437,...,-0.03192,-0.075847,-0.038993,-0.081923,-0.053173,-0.087037,-0.054137,-0.081888,-0.043033,-0.093889
leucine_200,-0.080804,-0.109326,-0.06159,-0.097883,-0.081317,-0.110382,-0.041176,-0.079492,-0.054277,-0.075481,...,-0.024384,-0.063989,-0.034492,-0.079504,-0.050372,-0.079121,-0.046078,-0.068018,-0.025788,-0.087582


In [166]:
FreshMedia_ColMed

,fm1-1,fm1-2,fm1-3,fm1-4,fm1-5,fm1-6,fm2-1,fm2-2,fm2-3,fm2-4,...,fm3-3,fm3-4,fm3-5,fm3-6,fm4-1,fm4-2,fm4-3,fm4-4,fm4-5,fm4-6
lactate_233,3.73007,3.996875,5.1694,3.469249,3.833251,3.531825,3.601907,3.260676,3.41797,3.350917,...,3.50528,3.538492,3.494285,3.024377,3.544946,3.243759,3.301766,4.048403,3.064937,3.350856
lactate_261,5.557978,5.962602,7.771688,5.146675,5.709604,5.219988,5.384089,4.840564,5.041231,4.976139,...,5.236993,5.268496,5.197997,4.471405,5.32181,4.81407,4.899662,6.091046,4.53813,4.982516
alanine_232,0.413932,0.415643,0.414151,0.41463,0.412151,0.410646,0.412663,0.413762,0.41379,0.41369,...,0.412341,0.413117,0.414044,0.413939,0.413974,0.412079,0.412088,0.410163,0.412207,0.414439
alanine_260,0.409213,0.410296,0.409472,0.41188,0.414769,0.411481,0.412412,0.412954,0.412067,0.411487,...,0.409644,0.413758,0.409579,0.407858,0.411768,0.409102,0.411156,0.410069,0.412239,0.412196
glycine_246,0.845163,0.839601,0.839686,0.835786,0.840496,0.83607,0.856871,0.83884,0.835007,0.834241,...,0.841977,0.840823,0.841145,0.844672,0.853852,0.844815,0.834647,0.841133,0.836773,0.837438
glycine_218,0.815191,0.801551,0.805108,0.805656,0.80269,0.806979,0.822257,0.804685,0.805897,0.800598,...,0.814981,0.810901,0.80666,0.80758,0.820948,0.809698,0.804249,0.798292,0.806215,0.80743
valine_288,1.368365,1.383349,1.390405,1.383692,1.388935,1.383296,1.354932,1.381897,1.38644,1.38031,...,1.388576,1.39379,1.393547,1.382063,1.364126,1.38325,1.384922,1.395602,1.387164,1.386874
valine_260,1.320215,1.36536,1.358352,1.355837,1.354448,1.35399,1.326399,1.353815,1.35642,1.357544,...,1.355578,1.35469,1.35509,1.356279,1.330268,1.357719,1.354417,1.36128,1.351464,1.35036
leucine_274,1.27205,1.278624,1.290044,1.276077,1.279585,1.279809,1.280724,1.274214,1.272318,1.280459,...,1.283831,1.279585,1.284347,1.279585,1.27357,1.280691,1.2794,1.290809,1.268586,1.283715
leucine_200,1.274436,1.289372,1.300799,1.279472,1.291531,1.28206,1.2771,1.274172,1.282369,1.281453,...,1.284457,1.288353,1.282606,1.282133,1.283813,1.285679,1.279042,1.296423,1.269399,1.285844


In [139]:
Ratios

,fm1-1,fm1-2,fm1-3,fm1-4,fm1-5,fm1-6,fm2-1,fm2-2,fm2-3,fm2-4,...,sm71,sm72,sm73,sm74,sm75,sm76,sm77,sm78,sm79,sm80
lactate_233,3.722559,3.939742,5.269562,3.345109,3.791052,3.446107,3.677053,3.264072,3.390013,3.366215,...,6.801215,6.299899,5.98019,7.093186,7.14671,6.075735,6.645294,5.921626,7.670855,7.692656
lactate_261,5.546786,5.87737,7.922271,4.962512,5.646748,5.093298,5.496416,4.845605,4.999997,4.998857,...,10.147332,9.34506,8.815188,10.45257,10.625476,8.908226,9.816182,8.68077,11.305839,11.336496
alanine_232,0.413098,0.409702,0.422176,0.399794,0.407614,0.40068,0.421272,0.414193,0.410405,0.415578,...,0.576039,0.645232,0.590139,0.643288,0.597151,0.664046,0.571438,0.635082,0.617822,0.710355
alanine_260,0.408389,0.404431,0.417406,0.397142,0.410203,0.401495,0.421016,0.413384,0.408697,0.413366,...,0.554038,0.634508,0.577078,0.639532,0.571628,0.644577,0.551848,0.612089,0.606645,0.681469
glycine_246,0.843461,0.827599,0.855956,0.805879,0.831244,0.815778,0.874748,0.839713,0.828177,0.838049,...,0.76356,0.738813,0.801734,0.743646,0.695059,0.689129,0.691249,0.654741,0.722282,0.676851
glycine_218,0.813549,0.790094,0.820708,0.776827,0.793854,0.787393,0.839412,0.805523,0.799305,0.804253,...,0.744224,0.704789,0.772669,0.723137,0.674318,0.671718,0.658108,0.645311,0.706517,0.650533
valine_288,1.36561,1.363574,1.417345,1.33418,1.373645,1.349723,1.383199,1.383336,1.375099,1.386612,...,1.425766,1.425766,1.520479,1.445,1.423302,1.453307,1.376138,1.374344,1.473013,1.461575
valine_260,1.317556,1.345843,1.384671,1.307321,1.339537,1.321128,1.354071,1.355224,1.345326,1.363742,...,1.390274,1.388003,1.470331,1.414841,1.384444,1.42483,1.34449,1.342282,1.440513,1.431292
leucine_274,1.269488,1.260347,1.31504,1.230415,1.265498,1.248747,1.307444,1.275541,1.261911,1.286305,...,1.233334,1.200544,1.303436,1.210262,1.213785,1.216391,1.175095,1.154462,1.259555,1.213751
leucine_200,1.27187,1.270941,1.326003,1.233688,1.277313,1.250944,1.303744,1.275499,1.27188,1.287304,...,1.244835,1.216459,1.31247,1.216848,1.220614,1.228646,1.186753,1.171784,1.281298,1.224402
